In [ ]:
# TABU SEARCH for 1 || ∑wᵢTᵢ Scheduling Problem

# Updated input data
jobs = list(range(1, 11))
processing_time = {1:7, 2:6, 3:6, 4:2, 5:2, 6:4, 7:7, 8:7, 9:5, 10:3}
due_date =         {1:24,2:21,3:23,4:14,5:35,6:16,7:31,8:11,9:13,10:17}
weight =           {1:6, 2:7, 3:10,4:7, 5:8, 6:12,7:11,8:17,9:12,10:11}

initial_sequence = [9, 3, 10, 7, 1, 6, 5, 4, 2, 8]

# Objective function: calculate total weighted tardiness
def calculate_total_weighted_tardiness(sequence):
    time = 0
    total = 0
    for job in sequence:
        time += processing_time[job]
        tardiness = max(0, time - due_date[job])
        total += weight[job] * tardiness
    return total

# Generate neighbors by adjacent swaps
def generate_neighbors(sequence):
    neighbors = []
    for i in range(len(sequence) - 1):
        neighbor = sequence.copy()
        neighbor[i], neighbor[i+1] = neighbor[i+1], neighbor[i]
        neighbors.append((i, i+1, neighbor))
    return neighbors

# Parameters
MAX_ITER = 85
TABU_TENURE = 5
ASPIRATION_IMPROVEMENT = 0.02  # 2% improvement threshold

# Tabu Search algorithm
from collections import deque

def tabu_search():
    current_sequence = initial_sequence.copy()
    current_value = calculate_total_weighted_tardiness(current_sequence)
    best_sequence = current_sequence.copy()
    best_value = current_value

    tabu_list = deque(maxlen=TABU_TENURE)

    for it in range(MAX_ITER):
        neighbors = generate_neighbors(current_sequence)
        best_neighbor = None
        best_neighbor_value = float('inf')
        best_move = None

        for i, j, neighbor in neighbors:
            move = (min(current_sequence[i], current_sequence[j]), max(current_sequence[i], current_sequence[j]))
            value = calculate_total_weighted_tardiness(neighbor)
            is_tabu = move in tabu_list

            # Apply aspiration criterion if improvement is ≥ 2%
            if not is_tabu or (is_tabu and value < best_value * (1 - ASPIRATION_IMPROVEMENT)):
                if is_tabu and value < best_value * (1 - ASPIRATION_IMPROVEMENT):
                    print(f"ASPIRATION TRIGGERED at Iter {it+1}: move {move} with value {value} (Best so far: {best_value})")

                if value < best_neighbor_value:
                    best_neighbor_value = value
                    best_neighbor = neighbor
                    best_move = move

        if best_neighbor is None:
            break  # No acceptable neighbor found

        current_sequence = best_neighbor
        current_value = best_neighbor_value
        tabu_list.append(best_move)

        if current_value < best_value:
            best_value = current_value
            best_sequence = current_sequence

        print(f"Iter {it+1}: Value = {current_value}, Best = {best_value}")

    return best_sequence, best_value

# Run the Tabu Search
final_seq, final_val = tabu_search()

print("\nFinal Sequence:", final_seq)
print("Final Objective (∑ wᵢTᵢ):", final_val)
